<a href="https://colab.research.google.com/github/ArpitaChatterjee/Comedian-Routine-Analysis/blob/main/Topic_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Topic Modeling

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Introduction

 The ultimate goal of topic modeling is to find various topics that are present in the corpus. Each document in the corpus will be made up of at least one topic, if not multiple topics.

Here, I'll be using  **Latent Dirichlet Allocation (LDA)**, which is one of many topic modeling techniques. It was specifically designed for text data.

To use a topic modeling technique, i need  (1) a document-term matrix and (2) the number of topics you would like the algorithm to pick up.

## Topic Modeling - Attempt #1 (All Text)

In [3]:
# Let's read in our document-term matrix
import pandas as pd
import pickle

data = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/NLP/dtm_stop.pkl')
data

,aaaaah,aaaaahhhhhhh,aaaaauuugghhhhhh,aaaahhhhh,aaah,aah,abc,abcs,ability,abject,able,ablebodied,abortion,abortions,abroad,absofuckinglutely,absolute,absolutely,absorb,absorbing,abstain,abuse,abused,academically,acc,accent,accents,accept,acceptable,acceptance,accepted,access,accident,accidentally,accomplished,accomplishments,according,accordingly,accoutrements,accurate,...,yellow,yelp,yep,yes,yesterday,yo,yoda,yoga,yogurt,yokels,yoko,yolanda,yolo,yooo,york,youd,youi,youll,young,younger,youngest,youth,youthis,youtube,youve,yulin,yummy,yyou,ze,zealand,zee,zen,zeppelin,zero,zillion,zombie,zombies,zoning,zoo,éclair
ali,0,0,0,0,0,0,1,0,0,0,2,0,0,0,0,0,0,0,0,1,0,1,2,0,0,0,0,0,0,0,0,0,1,1,0,0,2,0,0,0,...,0,1,0,9,0,0,0,1,0,0,1,0,1,0,0,0,1,4,2,0,0,0,0,0,2,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
anthony,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,6,0,0,0,0,10,0,0,0,0,0,0,0,0,0,0
bill,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,3,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2,0,0,0,0,0,0,...,1,0,0,3,0,0,0,1,0,0,0,0,0,0,1,1,0,5,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,1,1,1,1,1,0,0
bo,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,3,0,0,2,0,0,0,1,0,0,0,0,0,0,0,2,1,0,0,1,0,0,0,...,0,0,0,5,0,5,0,0,0,0,0,0,0,1,0,2,0,2,5,0,0,0,0,0,9,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
dave,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,7,0,1,0,0,0,0,0,0,0,0,1,7,0,3,10,0,0,0,1,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
hasan,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,...,0,0,0,4,0,4,1,1,3,1,0,0,1,0,12,2,0,3,0,1,0,0,0,0,8,0,0,0,0,0,2,1,0,1,0,0,0,0,0,0
jim,0,0,0,0,0,0,0,0,0,0,1,2,0,0,3,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0,0,1,0,0,0,2,0,2,0,0,0,0,0,0,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
joe,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0,0,4,0,1,0,0,3,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
john,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,2,0,0,1,0,0,0,1,0,...,1,0,0,11,0,0,0,0,0,0,0,0,0,0,7,2,0,1,2,1,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
louis,0,0,0,0,0,3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,1,0,0,0,0,0,0,1,...,0,0,1,12,0,0,0,0,0,0,0,0,0,0,4,2,0,1,8,3,0,1,0,1,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0


In [4]:
# Import the necessary modules for LDA with gensim
# Terminal / Anaconda Navigator: conda install -c conda-forge gensim
from gensim import matutils, models
import scipy.sparse

# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [5]:
# One of the required inputs is a term-document matrix
tdm = data.transpose()
tdm.head()

,ali,anthony,bill,bo,dave,hasan,jim,joe,john,louis,mike,ricky
aaaaah,0,0,1,0,0,0,0,0,0,0,0,0
aaaaahhhhhhh,0,0,0,1,0,0,0,0,0,0,0,0
aaaaauuugghhhhhh,0,0,0,1,0,0,0,0,0,0,0,0
aaaahhhhh,0,0,0,1,0,0,0,0,0,0,0,0
aaah,0,0,0,0,1,0,0,0,0,0,0,0


In [6]:
# We're going to put the term-document matrix into a new gensim format, from df --> sparse matrix --> gensim corpus
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [8]:
# Gensim also requires dictionary of the all terms and their respective location in the term-document matrix
cv = pickle.load(open("/content/drive/MyDrive/Colab Notebooks/NLP/cv_stop.pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.20.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term), we need to specify two other parameters - the number of topics and the number of passes. start the number of topics at 2, see if the results make sense, and increase the number from there.

In [9]:
# Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term),
# we need to specify two other parameters as well - the number of topics and the number of passes
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10)
lda.print_topics()

[(0,
  '0.008*"fucking" + 0.006*"say" + 0.006*"going" + 0.006*"want" + 0.005*"went" + 0.005*"day" + 0.004*"love" + 0.004*"thing" + 0.004*"hes" + 0.004*"didnt"'),
 (1,
  '0.008*"shit" + 0.007*"fuck" + 0.006*"fucking" + 0.005*"theyre" + 0.005*"cause" + 0.005*"man" + 0.005*"hes" + 0.004*"didnt" + 0.004*"really" + 0.004*"life"')]

In [10]:
# LDA for num_topics = 3
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=10)
lda.print_topics()

[(0,
  '0.008*"fucking" + 0.007*"say" + 0.006*"going" + 0.006*"fuck" + 0.005*"theyre" + 0.005*"shit" + 0.005*"want" + 0.005*"didnt" + 0.004*"did" + 0.004*"hes"'),
 (1,
  '0.007*"shit" + 0.006*"good" + 0.006*"fuck" + 0.005*"hes" + 0.005*"fucking" + 0.005*"didnt" + 0.005*"man" + 0.005*"life" + 0.005*"love" + 0.004*"want"'),
 (2,
  '0.007*"fucking" + 0.007*"went" + 0.006*"day" + 0.005*"cause" + 0.005*"shit" + 0.005*"going" + 0.004*"thing" + 0.004*"really" + 0.004*"theyre" + 0.004*"little"')]

In [ ]:
# LDA for num_topics = 4
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=10)
lda.print_topics()

These topics aren't looking too great. We've tried modifying our parameters. Let's try modifying our terms list as well.

## Topic Modeling - Attempt #2 (Nouns Only)

One popular trick is to look only at terms that are from one part of speech (only nouns, only adjectives, etc.). Check out the UPenn tag set: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html.

In [11]:
# Let's create a function to pull out nouns from a string of text
from nltk import word_tokenize, pos_tag

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

In [13]:
# Read in the cleaned data, before the CountVectorizer step
data_clean = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/NLP/data_clean.pkl')
data_clean

,transcript
ali,ladies and gentlemen please welcome to the sta...
anthony,thank you thank you thank you san francisco th...
bill,all right thank you thank you very much thank...
bo,bo what old macdonald had a farm e i e i o and...
dave,this is dave he tells dirty jokes for a living...
hasan,whats up davis whats up im home i had to bri...
jim,ladies and gentlemen please welcome to the ...
joe,ladies and gentlemen welcome joe rogan wha...
john,all right petunia wish me luck out there you w...
louis,introfade the music out lets roll hold there l...


In [15]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [17]:
 nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [18]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns = pd.DataFrame(data_clean.transcript.apply(nouns))
data_nouns

,transcript
ali,ladies gentlemen stage ali hi thank hello na s...
anthony,thank thank people i em i francisco city world...
bill,thank thank pleasure georgia area oasis i june...
bo,macdonald farm e i o farm pig e i i snort macd...
dave,jokes living stare work profound train thought...
hasan,whats davis whats home i netflix la york i son...
jim,ladies gentlemen stage mr jim jefferies thank ...
joe,ladies gentlemen joe fuck thanks phone fuckfac...
john,petunia thats hello hello chicago thank crowd ...
louis,music lets lights lights thank i i place place...


In [19]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
                  'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

# Recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words=stop_words)
data_cvn = cvn.fit_transform(data_nouns.transcript)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names())
data_dtmn.index = data_nouns.index
data_dtmn

,aaaaahhhhhhh,aaaaauuugghhhhhh,aaaahhhhh,aah,abc,abcs,ability,abortion,abortions,abuse,acc,accent,accents,acceptance,access,accident,accomplishments,accoutrements,accutane,achievement,acid,acquisitions,act,acting,actions,activity,actor,actors,ad,adam,adams,address,adjectives,adjustments,admirers,ads,adult,adults,advance,advantage,...,xmen,xray,ya,yall,yard,yards,yay,yeahmy,year,years,yecch,yelp,yep,yes,yesterday,yo,yoda,yoga,yogurt,yokels,yoko,yolo,york,youd,youll,youth,youthis,youtube,youve,yulin,yummy,ze,zealand,zee,zeppelin,zillion,zombie,zombies,zoo,éclair
ali,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,7,5,0,1,0,1,0,0,0,0,0,0,1,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
anthony,0,0,0,0,0,0,0,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,7,5,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,10,0,0,0,0,0,0,0
bill,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,6,0,0,0,...,0,0,5,0,1,0,0,0,3,17,0,0,0,1,0,0,0,1,0,0,0,0,1,0,3,0,0,1,0,0,0,1,0,0,0,1,1,1,0,0
bo,1,1,1,0,0,0,1,0,0,0,1,0,0,0,0,2,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,4,0,0,0,2,0,1,0,0,0,0,0,0,0,1,1,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0
dave,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,1,0,2,9,0,0,0,3,0,0,0,0,0,0,0,0,1,4,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0
hasan,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,5,12,0,0,0,0,0,2,1,1,2,1,0,1,12,0,1,0,0,0,5,0,0,0,0,1,0,0,0,0,0,0
jim,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,1,1,0,0,3,2,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,3,6,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,7,0,0,0,0,0,0,0,0,0,0,0
joe,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,3,6,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0
john,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,1,0,2,11,0,0,0,4,0,0,0,0,0,0,0,0,7,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1
louis,0,0,0,3,0,0,0,0,0,0,0,0,0,0,2,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,3,4,1,0,0,5,0,0,0,0,0,0,0,0,4,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [20]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [21]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.008*"man" + 0.007*"shit" + 0.007*"way" + 0.007*"hes" + 0.007*"day" + 0.006*"life" + 0.006*"thing" + 0.006*"fuck" + 0.006*"cause" + 0.005*"lot"'),
 (1,
  '0.012*"thing" + 0.011*"day" + 0.009*"life" + 0.008*"shit" + 0.008*"guy" + 0.007*"cause" + 0.007*"hes" + 0.006*"fuck" + 0.006*"gon" + 0.006*"women"')]

In [22]:
# Let's try topics = 3
ldan = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.007*"shit" + 0.007*"cause" + 0.006*"man" + 0.006*"way" + 0.006*"point" + 0.006*"night" + 0.006*"guy" + 0.005*"lot" + 0.005*"didnt" + 0.005*"kind"'),
 (1,
  '0.012*"day" + 0.010*"thing" + 0.009*"shit" + 0.008*"man" + 0.007*"fuck" + 0.007*"cause" + 0.007*"hes" + 0.007*"way" + 0.006*"gon" + 0.006*"life"'),
 (2,
  '0.012*"life" + 0.008*"hes" + 0.008*"dad" + 0.007*"thing" + 0.007*"man" + 0.006*"shes" + 0.006*"way" + 0.006*"stuff" + 0.005*"shit" + 0.005*"bo"')]

In [23]:
# Let's try 4 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.009*"day" + 0.007*"thing" + 0.007*"hes" + 0.007*"years" + 0.007*"joke" + 0.006*"shit" + 0.006*"lot" + 0.006*"god" + 0.005*"id" + 0.005*"woman"'),
 (1,
  '0.010*"day" + 0.008*"thing" + 0.008*"cause" + 0.006*"way" + 0.006*"guy" + 0.006*"stuff" + 0.006*"man" + 0.006*"women" + 0.005*"point" + 0.005*"hes"'),
 (2,
  '0.011*"dad" + 0.009*"fuck" + 0.009*"life" + 0.008*"day" + 0.008*"shit" + 0.008*"house" + 0.007*"man" + 0.007*"school" + 0.007*"mom" + 0.006*"shes"'),
 (3,
  '0.012*"shit" + 0.010*"thing" + 0.010*"life" + 0.009*"hes" + 0.009*"man" + 0.008*"guy" + 0.008*"way" + 0.007*"cause" + 0.007*"day" + 0.007*"gon"')]

## Topic Modeling - Attempt #3 (Nouns and Adjectives)

In [24]:
# Let's create a function to pull out nouns from a string of text
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

In [25]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns_adj = pd.DataFrame(data_clean.transcript.apply(nouns_adj))
data_nouns_adj

,transcript
ali,ladies gentlemen welcome stage ali wong hi wel...
anthony,thank san francisco thank good people surprise...
bill,right thank thank pleasure greater atlanta geo...
bo,old macdonald farm e i i o farm pig e i i snor...
dave,dirty jokes living stare most hard work profou...
hasan,whats davis whats im home i netflix special la...
jim,ladies gentlemen welcome stage mr jim jefferie...
joe,ladies gentlemen joe fuck san francisco thanks...
john,right petunia august thats good right hello he...
louis,music lets lights lights thank much i i i nice...


In [26]:
# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
cvna = CountVectorizer(stop_words=stop_words, max_df=.8)
data_cvna = cvna.fit_transform(data_nouns_adj.transcript)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names())
data_dtmna.index = data_nouns_adj.index
data_dtmna

,aaaaah,aaaaahhhhhhh,aaaaauuugghhhhhh,aaaahhhhh,aah,abc,abcs,ability,abject,able,ablebodied,abortion,abortions,absolute,abuse,acc,accent,accents,acceptable,acceptance,access,accident,accomplished,accomplishments,accoutrements,accurate,accutane,achievement,acid,acoustic,acquisitions,act,acting,actions,active,activity,actor,actors,actual,ad,...,yard,yards,yay,yeahmy,yecch,yellow,yelp,yep,yes,yesterday,yo,yoda,yoga,yogurt,yokels,yoko,yolo,york,youd,youll,young,younger,youngest,youth,youthis,youtube,youve,yulin,yummy,yyou,ze,zealand,zee,zeppelin,zero,zillion,zombie,zombies,zoo,éclair
ali,0,0,0,0,0,1,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,2,2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
anthony,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,10,0,0,0,0,0,0,0,0
bill,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,3,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,1,0,0
bo,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,2,0,2,0,0,0,0,0,0,0,1,1,5,0,0,0,0,0,4,0,0,0,0,0,0,0,1,0,0,0,0,0
dave,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,1,0,0,0,0,1,0,...,0,1,1,0,0,0,0,0,3,0,1,0,0,0,0,0,0,1,4,1,10,0,0,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0
hasan,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,4,1,1,2,1,0,1,12,0,1,0,1,0,0,0,0,5,0,0,0,0,0,2,0,0,0,0,0,0,0
jim,0,0,0,0,0,0,0,0,0,1,2,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,3,2,1,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0
joe,0,0,0,0,0,0,0,0,0,2,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,4,0,1,0,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0
john,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,1,0,1,0,2,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,...,0,0,2,0,0,1,0,0,4,0,0,0,0,0,0,0,0,7,0,1,2,1,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,1
louis,0,0,0,0,3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,3,0,...,0,0,0,0,1,0,0,0,5,0,0,0,0,0,0,0,0,4,1,0,8,3,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [28]:
# Let's start with 2 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=2, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.006*"joke" + 0.003*"jokes" + 0.003*"dead" + 0.003*"bo" + 0.003*"comedy" + 0.003*"jenny" + 0.002*"hell" + 0.002*"parents" + 0.002*"repeat" + 0.002*"nuts"'),
 (1,
  '0.005*"mom" + 0.003*"ass" + 0.003*"parents" + 0.003*"son" + 0.003*"door" + 0.003*"wife" + 0.003*"dick" + 0.003*"hasan" + 0.002*"friend" + 0.002*"clinton"')]

In [29]:
# Let's try 3 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.004*"joke" + 0.003*"jenny" + 0.003*"ahah" + 0.003*"guns" + 0.003*"son" + 0.003*"class" + 0.003*"friend" + 0.003*"ass" + 0.003*"girlfriend" + 0.003*"door"'),
 (1,
  '0.005*"joke" + 0.005*"ok" + 0.004*"bo" + 0.004*"mom" + 0.004*"eye" + 0.003*"repeat" + 0.003*"jokes" + 0.003*"anthony" + 0.003*"contact" + 0.003*"husband"'),
 (2,
  '0.007*"mom" + 0.005*"parents" + 0.004*"hasan" + 0.004*"clinton" + 0.004*"wife" + 0.003*"cow" + 0.003*"york" + 0.003*"gun" + 0.003*"brown" + 0.003*"door"')]

In [30]:
# Let's try 4 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.009*"ok" + 0.006*"tit" + 0.005*"husband" + 0.004*"older" + 0.004*"date" + 0.004*"pregnant" + 0.004*"worst" + 0.004*"ha" + 0.004*"murder" + 0.004*"toilet"'),
 (1,
  '0.005*"joke" + 0.004*"ahah" + 0.004*"gay" + 0.003*"nuts" + 0.003*"son" + 0.003*"hell" + 0.003*"nigga" + 0.003*"wife" + 0.003*"gun" + 0.003*"rape"'),
 (2,
  '0.007*"mom" + 0.006*"parents" + 0.004*"friend" + 0.004*"joke" + 0.003*"hasan" + 0.003*"door" + 0.003*"clinton" + 0.003*"jenny" + 0.003*"anthony" + 0.003*"york"'),
 (3,
  '0.007*"bo" + 0.006*"repeat" + 0.006*"guns" + 0.006*"um" + 0.005*"eye" + 0.005*"contact" + 0.004*"ass" + 0.004*"cos" + 0.004*"girlfriend" + 0.004*"brain"')]

## Identify Topics in Each Document

Out of the 9 topic models we looked at, the nouns and adjectives, 4 topic one made the most sense. So let's pull that down here and run it through some more iterations to get more fine-tuned topics.

In [31]:
# Our final LDA model (for now)
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=80)
ldana.print_topics()

[(0,
  '0.005*"mom" + 0.004*"joke" + 0.004*"parents" + 0.003*"ok" + 0.003*"hasan" + 0.003*"dick" + 0.003*"anthony" + 0.003*"gun" + 0.003*"door" + 0.002*"mad"'),
 (1,
  '0.005*"joke" + 0.005*"clinton" + 0.004*"wife" + 0.004*"ahah" + 0.004*"mom" + 0.003*"gay" + 0.003*"cow" + 0.003*"friend" + 0.003*"nigga" + 0.003*"nuts"'),
 (2,
  '0.010*"jenny" + 0.005*"accident" + 0.004*"scrambler" + 0.004*"friend" + 0.004*"argument" + 0.004*"andy" + 0.003*"morning" + 0.003*"club" + 0.003*"parents" + 0.003*"flight"'),
 (3,
  '0.008*"bo" + 0.006*"repeat" + 0.006*"guns" + 0.006*"um" + 0.005*"eye" + 0.005*"contact" + 0.004*"ass" + 0.004*"cos" + 0.004*"girlfriend" + 0.004*"brain"')]

These four topics look pretty decent. I'll on these for now.
* Topic 0: mom, parents
* Topic 1: husband, wife
* Topic 2: guns
* Topic 3: profanity

In [32]:
# Let's take a look at which topics each transcript contains
corpus_transformed = ldana[corpusna]
list(zip([a for [(a,b)] in corpus_transformed], data_dtmna.index))

[(0, 'ali'),
 (0, 'anthony'),
 (0, 'bill'),
 (3, 'bo'),
 (1, 'dave'),
 (0, 'hasan'),
 (3, 'jim'),
 (0, 'joe'),
 (1, 'john'),
 (0, 'louis'),
 (2, 'mike'),
 (1, 'ricky')]

For a first pass of LDA, these kind of make sense to me, for now.
* Topic 0: mom, parents [Anthony, Hasan, Louis, Ricky]
* Topic 1: husband, wife [Ali, John, Mike]
* Topic 2: guns [Bill, Bo, Jim]
* Topic 3: profanity [Dave, Joe]